In [5]:
import pandas as pd
import numpy as np
from datetime import datetime

In [6]:
data=pd.read_csv('GEFCom2014N.csv').dropna()
data['load']=data['load']/10000
data['T']=data['T']/100
# Convert timestamp to datetime format
data['Datetime'] = data['Datetime'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

# Set the datetime as index
data.set_index('Datetime', inplace=True)

# Resample the data to hourly frequency
data = data.resample('H').mean()

# Create lag features
for i in range(1, 25):
    data['lag_{}'.format(i)] = data['load'].shift(i)

# Create rolling mean and standard deviation features
data['rolling_mean'] = data['load'].rolling(window=24).mean()
data['rolling_std'] = data['load'].rolling(window=24).std()

# Create weekday and hour features
data['weekday'] = data.index.weekday
data['hour'] = data.index.hour

# Remove missing values
data.dropna(inplace=True)

# Split the data into train and test sets
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]

In [8]:
#CNN-LSTM
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten

# Prepare the train and test data
X_train, y_train = train_data.drop('load', axis=1), train_data['load']
X_test, y_test = test_data.drop('load', axis=1), test_data['load']

# Reshape the data for CNN-LSTM
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define the model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(64, return_sequences=True))
model.add(Flatten())
model.add(Dense(1))

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
mae = history.history['val_loss'][-1]
mse = mean_squared_error(y_test, model.predict(X_test))
rmse = np.sqrt(mse)

ypred=model.predict(X_test)
y_test=test_data['load']
y_cnnlstm=ypred
#For MAE
mae = np.mean(np.abs(ypred.ravel() - y_test.ravel()))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
mape = np.mean(np.abs((y_test.ravel() - ypred.ravel()) / y_test.ravel())) * 100
print('MAPE = {:.2f} (%)'.format(mape))


# For RMSE
from sklearn.metrics import mean_squared_error 
mse=mean_squared_error(y_test,ypred) 
rsme=np.sqrt(mse) 
print('RSME = {:.2f} (%)'.format(rsme))


Epoch 1/50
986/986 [==============================] - 18s 15ms/step - loss: 0.0013 - val_loss: 1.5612e-04
Epoch 2/50
986/986 [==============================] - 15s 15ms/step - loss: 1.4940e-04 - val_loss: 1.1800e-04
Epoch 3/50
986/986 [==============================] - 15s 15ms/step - loss: 1.1544e-04 - val_loss: 8.1008e-05
Epoch 4/50
986/986 [==============================] - 15s 15ms/step - loss: 8.7346e-05 - val_loss: 5.8603e-05
Epoch 5/50
986/986 [==============================] - 15s 15ms/step - loss: 7.1810e-05 - val_loss: 5.8870e-05
Epoch 6/50
986/986 [==============================] - 15s 15ms/step - loss: 6.0178e-05 - val_loss: 4.1026e-05
Epoch 7/50
986/986 [==============================] - 15s 15ms/step - loss: 5.8076e-05 - val_loss: 3.8838e-05
Epoch 8/50
986/986 [==============================] - 15s 15ms/step - loss: 5.3239e-05 - val_loss: 5.5611e-05
Epoch 9/50
986/986 [==============================] - 15s 15ms/step - loss: 4.7155e-05 - val_loss: 3.4686e-05
Epoch 10/50
98